In [ ]:
#from lxml import etree
from copy import deepcopy
from io import BytesIO
import os, glob, math
import re, sys, json, time, os, six, getpass, requests, cv2
import dask
import pandas as pd
import numpy as np
from IPython.display import display

def recursive_dict(element):
     return element.tag, \
            dict(map(recursive_dict, element)) or element.text
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

#https://blog.paperspace.com/data-augmentation-for-object-detection-rotation-and-shearing/
def get_corners(bboxes):
    """Get corners of bounding boxes
    Parameters
    ----------
    bboxes: numpy.ndarray
        Numpy array containing bounding boxes of shape `N X 4` where N is the 
        number of bounding boxes and the bounding boxes are represented in the
        format `x1 y1 x2 y2`
    returns
    -------
    numpy.ndarray
        Numpy array of shape `N x 8` containing N bounding boxes each described by their 
        corner co-ordinates `x1 y1 x2 y2 x3 y3 x4 y4`  
    """
    width = (bboxes[:,2] - bboxes[:,0]).reshape(-1,1)
    height = (bboxes[:,3] - bboxes[:,1]).reshape(-1,1)    
    x1 = bboxes[:,0].reshape(-1,1)
    y1 = bboxes[:,1].reshape(-1,1)
    x2 = x1 + width
    y2 = y1 
    x3 = x1
    y3 = y1 + height
    x4 = bboxes[:,2].reshape(-1,1)
    y4 = bboxes[:,3].reshape(-1,1)
    corners = np.hstack((x1,y1,x2,y2,x3,y3,x4,y4))
    return corners

def rotate_box(corners, angle, cx, cy, h, w):
    """Rotate the bounding box.
    Parameters
    ----------
    corners : numpy.ndarray
        Numpy array of shape `N x 8` containing N bounding boxes each described by their 
        corner co-ordinates `x1 y1 x2 y2 x3 y3 x4 y4`
    angle : float
        angle by which the image is to be rotated
    cx : int
        x coordinate of the center of image (about which the box will be rotated)
    cy : int
        y coordinate of the center of image (about which the box will be rotated)
    h : int 
        height of the image
    w : int 
        width of the image
    Returns
    -------
    numpy.ndarray
        Numpy array of shape `N x 8` containing N rotated bounding boxes each described by their 
        corner co-ordinates `x1 y1 x2 y2 x3 y3 x4 y4`
    """

    corners = corners.reshape(-1,2)
    corners = np.hstack((corners, np.ones((corners.shape[0],1), dtype = type(corners[0][0]))))
    M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cx
    M[1, 2] += (nH / 2) - cy
    # Prepare the vector to be transformed
    calculated = np.dot(M,corners.T).T
    calculated = calculated.reshape(-1,8)
    return calculated, nW, nH

def get_enclosing_box(corners):
    """Get an enclosing box for ratated corners of a bounding box
    Parameters
    ----------
    corners : numpy.ndarray
        Numpy array of shape `N x 8` containing N bounding boxes each described by their 
        corner co-ordinates `x1 y1 x2 y2 x3 y3 x4 y4`  
    Returns 
    -------
    numpy.ndarray
        Numpy array containing enclosing bounding boxes of shape `N X 4` where N is the 
        number of bounding boxes and the bounding boxes are represented in the
        format `x1 y1 x2 y2`
    """
    x_ = corners[:,[0,2,4,6]]
    y_ = corners[:,[1,3,5,7]]
    xmin = np.min(x_,1).reshape(-1,1)
    ymin = np.min(y_,1).reshape(-1,1)
    xmax = np.max(x_,1).reshape(-1,1)
    ymax = np.max(y_,1).reshape(-1,1)
    final = np.hstack((xmin, ymin, xmax, ymax,corners[:,8:]))
    return final


def rotate_get_new_cord(x, y=0, imgw=100, imgh=100, degree=0):
    """
    X=xcos(θ)+ysin(θ)
    Y=−xsin(θ)+ycos(θ)
    """
    largerOneOfWidthHeight = max([x,y])
    originX = imgw//2
    originY = imgh//2
    yNormalCoordinateSystem = imgh-y #Y軸轉換為一般座標系算法
    xOriginSetToImageCenter = x-originX #設定原點在圖中間
    yOriginSetToImageCenter = yNormalCoordinateSystem-originY
    radian = math.pi*(degree/180)
    xRotated = xOriginSetToImageCenter * math.cos(radian) - yOriginSetToImageCenter * math.sin(radian)
    yRotated = xOriginSetToImageCenter * math.sin(radian) * -1 + yOriginSetToImageCenter * math.cos(radian) #轉軸公式
    newWidth = imgw * math.cos(radian) + imgh * math.sin(radian)
    newHeight = imgw * math.sin(radian) + imgh * math.cos(radian)
    xOriginSetAwayFromImageCenter = xRotated+originX #unset原點在圖中間
    yOriginSetAwayFromImageCenter = yRotated+originY
    yComputerImageCoordinateSystem = imgh-yOriginSetAwayFromImageCenter #y軸轉換為電腦圖片算法
    xScaled = xOriginSetAwayFromImageCenter #/newWidth*imgw
    yScaled = yComputerImageCoordinateSystem #/newHeight*imgh
    return round(xScaled), round(yScaled), newWidth, newHeight

testbboxes = np.array([[0,0,100,100]]) #x1 y1 x2 y2
testwidth = 400
testheight = 100
#testTransformedBox = rotate_box()

if True:
    corners, newWidth, newHeight = rotate_box(get_corners(testbboxes), 90, testwidth//2, testheight//2, testheight, testwidth)
    display(np.round(corners), newWidth, newHeight) #x1 y1 x2 y2 x3 y3 x4 y4
    testenclosingbox = get_enclosing_box(corners) #x1 y1 x2 y2
    #display(testenclosingbox)
    display(np.round(testenclosingbox))
    #display(cv2.getRotationMatrix2D((cX, cY), -angle, 1.0))
#returns x1 y1 x2 y2 x3 y3 x4 y4
#display(rotate_get_new_cord(4000,0,testwidth,testheight,90))
#display(cv2.getRotationMatrix2D((testwidth//2, testheight//2), -90, 1.0).shape)
#(0,0) -> (100,0) -> (100,100) -> (0,100)

In [ ]:
rootpath = ''
newWebPathBase = ['/data/local-files/?d=coffee-beans/', 'http://localhost/coffeebeans/'][1]
replaceSourcePath = ''
resultDict = []
xmlFilePaths = {os.path.basename(f):f for f in glob.iglob(rootpath+'**/*.xml', recursive=True)}
jpgFilePaths = [f for f in glob.iglob(rootpath+ '**/*.jpg', recursive=True)]
print(len(jpgFilePaths))
print(len(xmlFilePaths))

In [ ]:
for jpgFilePath_i,jpgFilePath in enumerate(jpgFilePaths):
    print(jpgFilePath_i, jpgFilePath)
    singleImageDict = {}
    testMatchingXMLFileName = os.path.basename(jpgFilePath).replace('.jpg','.xml')
    predictionDictBase = {
        "type": "rectanglelabels",        
        "from_name": "label", "to_name": "image",
        "image_rotation": 0,
    }
    if testMatchingXMLFileName in xmlFilePaths.keys():
        xmlFilePath = xmlFilePaths[testMatchingXMLFileName]
        f = open(xmlFilePath, 'r', encoding='utf-8')
        xmlContent = f.read()
        f.close()
        tree = etree.fromstring(xmlContent)
        imageFilePath = list(tree.iter("path"))[0].text
        originalSize = recursive_dict(list(tree.iter("size"))[0])[1]
        sourceSizeDict = {"original_width": int(originalSize['width']), "original_height": int(originalSize['height'])}
        newImageFilePath = jpgFilePath.replace(rootpath,newWebPathBase).replace(replaceSourcePath,newWebPathBase).replace('\\','/')
        singleImageDict['data'] = {'image': newImageFilePath}
        needToRotate = True if sourceSizeDict['original_width']!=3024 else False
        originalSizeDict = {"original_width": 3024, "original_height": 4032} if needToRotate==True else sourceSizeDict
        predictionDictBase = {**predictionDictBase, **originalSizeDict}
        annotatedObjs = []
        angle = -90
        for keyi, element in enumerate(tree.iter("object")):
            elementDict = recursive_dict(element)
            if needToRotate==True:
                testbboxes = np.array([[int(elementDict[1]['bndbox']['xmin']),int(elementDict[1]['bndbox']['ymin']),int(elementDict[1]['bndbox']['xmax']),int(elementDict[1]['bndbox']['ymax'])]]) #x1 y1 x2 y2
                testwidth = sourceSizeDict['original_width']
                testheight = sourceSizeDict['original_height']
                corners, newImageWidth, newImageHeight = rotate_box(get_corners(testbboxes), angle, testwidth//2, testheight//2, testheight, testwidth)
                testenclosingbox = np.round(get_enclosing_box(corners))[0,:]
                newXminAxis = testenclosingbox[0]/newImageWidth*100
                newYminAxis = testenclosingbox[1]/newImageHeight*100
                newXmaxAxis = testenclosingbox[2]/newImageWidth*100
                newYmaxAxis = testenclosingbox[3]/newImageHeight*100
                newBBoxWidth = abs(newXmaxAxis-newXminAxis)
                newBBoxHeight = abs(newYmaxAxis-newYminAxis)
            else:
                newXminAxis = int(elementDict[1]['bndbox']['xmin'])/int(originalSize['width'])*100
                newYminAxis = int(elementDict[1]['bndbox']['ymin'])/int(originalSize['height'])*100
                newBBoxWidth = (int(elementDict[1]['bndbox']['xmax']) - int(elementDict[1]['bndbox']['xmin']))/int(originalSize['width'])*100
                newBBoxHeight = ( int(elementDict[1]['bndbox']['ymax']) - int(elementDict[1]['bndbox']['ymin']))/int(originalSize['height'])*100
            """
                {
                "id": "result2",
                "type": "rectanglelabels",        
                "from_name": "label", "to_name": "image",
                "original_width": 600, "original_height": 403,
                "image_rotation": 0,
                "value": {
                    "rotation": 0,          
                    "x": 75.47, "y": 82.33,
                    "width": 5.74, "height": 7.40,
                    "rectanglelabels": ["Car"]
                    }
                }
            """
            coffeeObjDict = (merge_two_dicts(
                predictionDictBase,
                {
                    'id': 'result'+str(keyi+1),
                    'value': {
                    "rotation": 0,
                    'x':newXminAxis,
                    'y':newYminAxis,
                    'width':newBBoxWidth,
                    'height':newBBoxHeight,
                    "rectanglelabels": [elementDict[1]['name']]
                }}
            ))
            annotatedObjs.append(coffeeObjDict)
        annotationAuthor = ['s110109@shsh.tw','Chen','YiChang',2] if re.search('IChang_bad',xmlFilePath)!=None else ['tingjhenjiang@gmail.com','Jiang','TingJhen',1]
        annotationAuthorDict = {
            "id":annotationAuthor[3],
            "email":annotationAuthor[0],
            "first_name":annotationAuthor[2],
            "last_name":annotationAuthor[1]
        }
        singleImageDict['annotations'] = [{'result':annotatedObjs}]
        singleImageDict['completed_by'] = annotationAuthorDict
    else:
        newImageFilePath = jpgFilePath.replace(rootpath,newWebPathBase).replace(replaceSourcePath,newWebPathBase).replace('\\','/')
        singleImageDict['data'] = {'image': newImageFilePath}
        annotationAuthor = [None]
    singleImageDict['id'] = jpgFilePath_i+1
    if (annotationAuthor[0] in [None]):
        resultDict.append(singleImageDict)
    else:
        next

In [ ]:
f = open(os.path.join(rootpath,'labels_labelstudio_format','tasks.json'), "w")
f.write(json.dumps(resultDict))
f.close()

In [ ]:
for xmlFilei,xmlFilePath in enumerate(xmlFilePaths.values()):
    f = open(xmlFilePath, 'r', encoding='utf-8')
    xmlContent = f.read()
    f.close()
    tree = etree.fromstring(xmlContent)
    singleImageDict = {}
    print(xmlFilePath)
    imageFilePath = list(tree.iter("path"))[0].text
    originalSize = recursive_dict(list(tree.iter("size"))[0])[1]
    originalSizeDict = {"original_width": originalSize['width'], "original_height": originalSize['height']}
    newImageFilePath = imageFilePath.replace(rootpath,newWebPathBase).replace(replaceSourcePath,newWebPathBase).replace('\\','/')
    singleImageDict['data'] = {'image': newImageFilePath}
    predictionDictBase = {**{
        "type": "rectanglelabels",        
        "from_name": "label", "to_name": "image",
        "image_rotation": 0,
    }, **originalSizeDict}
    annotatedObjs = []
    for keyi, element in enumerate(tree.iter("object")):
        elementDict = recursive_dict(element)
        """
            {
            "id": "result2",
            "type": "rectanglelabels",        
            "from_name": "label", "to_name": "image",
            "original_width": 600, "original_height": 403,
            "image_rotation": 0,
            "value": {
                "rotation": 0,          
                "x": 75.47, "y": 82.33,
                "width": 5.74, "height": 7.40,
                "rectanglelabels": ["Car"]
                }
            }
        """
        coffeeObjDict = (merge_two_dicts(
            predictionDictBase,
            {
                'id': 'result'+str(keyi+1),
                'value': {
                "rotation": 0,
                'x':int(elementDict[1]['bndbox']['xmin'])/int(originalSize['width'])*100,
                'y':int(elementDict[1]['bndbox']['ymin'])/int(originalSize['height'])*100,
                'width':(int(elementDict[1]['bndbox']['xmax']) - int(elementDict[1]['bndbox']['xmin']))
                    /int(originalSize['width'])*100,
                'height':( int(elementDict[1]['bndbox']['ymax']) - int(elementDict[1]['bndbox']['ymin']))
                    /int(originalSize['height'])*100,
                "rectanglelabels": [elementDict[1]['name']]
            }}
        ))
        annotatedObjs.append(coffeeObjDict)
        break
    singleImageDict['annotations'] = [{'result':annotatedObjs}]
    singleImageDict['id'] = xmlFilei+1
    resultDict.append(singleImageDict)
    break